# Instance Detection
Training:
1. Generate synthetic training data
2. Train FasterRCNN on synthetic training data

Testing:
1. Test FasterRCNN on Scene images

In [2]:
import logging
import os
import json
import random
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.data import build_detection_test_loader, DatasetCatalog, MetadataCatalog
from detectron2.config import get_cfg
from detectron2.engine import hooks
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.layers.nms import batched_nms
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.structures import Boxes, BoxMode, Instances
from detectron2.utils.logger import setup_logger

# import custom nerfdet utilities
from nerfdet.config import add_config
from nerfdet.utils.visualizer import ColorMode, Visualizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
setup_logger()

<Logger detectron2 (DEBUG)>

## Register InsDet Dataset for detectron2

In [3]:
def get_thing_classes(data_root, file_name="fg_object_id.json"):
    # get things classes and rename them from 0 to Number classes - 1

    json_file = os.path.join(data_root, file_name)
    thing_classes_dict = {}
    with open(json_file, mode='r') as f:
        fg_dict = json.load(f)

    thing_classes = []  # a list of noun names
    thing_classes_correspondence_dict = {}  # original noun class : relabeled noun class

    for object_name in fg_dict.keys():
        thing_classes_correspondence_dict[fg_dict[object_name]] = fg_dict[object_name]
        thing_classes.append(object_name)

    return thing_classes_correspondence_dict, thing_classes

In [4]:
dataset_root = os.path.join(os.path.dirname(os.getcwd()), 'NeRFDetExps/Data')
anno_root = os.path.join(dataset_root, 'annotations')

# train data
train_path = os.path.join(dataset_root, 'train')
train_json = os.path.join(anno_root, 'instances_train.json')
register_coco_instances('coco_realDB_train', {}, train_json, train_path)

# validation data
val_path = os.path.join(dataset_root, 'val')
val_json = os.path.join(anno_root, 'instances_val.json')
register_coco_instances('coco_realDB_val', {}, val_json, val_path)

# test data
test_path = os.path.join(dataset_root, 'test_8')
test_json = os.path.join(anno_root, 'instances_test_8.json')
register_coco_instances('coco_realDB_test', {}, test_json, test_path)

_, thing_classes = get_thing_classes(dataset_root)
MetadataCatalog.get("coco_realDB_train").thing_classes = thing_classes
MetadataCatalog.get("coco_realDB_val").thing_classes = thing_classes
MetadataCatalog.get("coco_realDB_test").thing_classes = thing_classes

### Common configs

In [5]:
# setup configs
# FasterRCNN architecture
yaml_f = 'faster_rcnn_R_50_FPN_3x.yaml'
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/{}".format(yaml_f)))
# load COCO pretrained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/{}".format(yaml_f))

# ROI Heads
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 100

# Solver
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.TEST.EVAL_PERIOD = 500
cfg.SOLVER.MAX_ITER = 2000

# Input
cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.MAX_SIZE_TRAIN = 1024
cfg.INPUT.MAX_SIZE_TEST = 1024
cfg.INPUT.MIN_SIZE_TRAIN = (768,)
cfg.INPUT.MIN_SIZE_TEST = 0
cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = 'choice'

## Train on FasterRCNN

In [5]:
# Datasets
cfg.DATASETS.TRAIN = ('coco_realDB_train',)
cfg.DATASETS.TEST = ('coco_realDB_val',)
cfg.DATALOADER.NUM_WORKERS = 0

# Output_dir
cfg.OUTPUT_DIR = "./exp_on_FasterRCNN"

# call default Trainer and Predictor for FasterRCNN
from nerfdet.engine.train_net import Trainer

# train on train_data with inference on val data
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
if cfg.TEST.AUG.ENABLED:
    trainer.register_hooks(
        [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]
    )
trainer.train()

[03/15 17:04:51 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[03/15 17:05:00 d2.data.datasets.coco]: Loading /home/SQQ/svid/NeRFDetExps/Data/annotations/instances_train.json takes 8.85 seconds.
[03/15 17:05:00 d2.data.datasets.coco]: Loaded 19191 images in COCO format from /home/SQQ/svid/NeRFDetExps/Data/annotations/instances_train.json
[03/15 17:05:04 d2.data.build]: Removed 0 images with no usable annotations. 19191 images left.
[03/15 17:05:04 d2.data.build]: Distribution of instances among all 100 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
| 000_aveda_s.. | 5775         | 001_binder_.. | 5758         | 002_binder_.. | 5734         |
| 003_bombik_.. | 5764         | 004_bonne_m.. | 5745         | 005_bonne_m.. | 5745         |
| 006_bonne_m.. | 5757         | 007_costa_c.. | 5741         | 008_essenti.. | 5771         |
| 009_garlic_.. | 5544         | 010_handcre.. | 5795         | 011

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (101, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (101,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (400, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (400,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{

[03/15 17:05:10 d2.engine.train_loop]: Starting training from iteration 0


/home/SQQ/anaconda3/envs/svid/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


...

[03/15 17:37:02 d2.evaluation.evaluator]: Inference done 2428/6397. Dataloading: 0.0084 s/iter. Inference: 0.0307 s/iter. Eval: 0.0003 s/iter. Total: 0.0394 s/iter. ETA=0:02:36
[03/15 17:37:07 d2.evaluation.evaluator]: Inference done 2558/6397. Dataloading: 0.0084 s/iter. Inference: 0.0306 s/iter. Eval: 0.0003 s/iter. Total: 0.0394 s/iter. ETA=0:02:31
[03/15 17:37:12 d2.evaluation.evaluator]: Inference done 2687/6397. Dataloading: 0.0084 s/iter. Inference: 0.0306 s/iter. Eval: 0.0003 s/iter. Total: 0.0394 s/iter. ETA=0:02:26
[03/15 17:37:17 d2.evaluation.evaluator]: Inference done 2814/6397. Dataloading: 0.0084 s/iter. Inference: 0.0307 s/iter. Eval: 0.0003 s/iter. Total: 0.0394 s/iter. ETA=0:02:21
[03/15 17:37:22 d2.evaluation.evaluator]: Inference done 2948/6397. Dataloading: 0.0084 s/iter. Inference: 0.0306 s/iter. Eval: 0.0003 s/iter. Total: 0.0393 s/iter. ETA=0:02:15
[03/15 17:37:27 d2.evaluation.evaluator]: Inference done 3069/6397. Dataloading: 0.0084 s/iter. Inference: 0.0306 s

[03/15 17:40:10 d2.evaluation.coco_evaluation]: Per-category bbox AP: 
| category                   | AP     | category                     | AP     | category                     | AP     |
|:---------------------------|:-------|:-----------------------------|:-------|:-----------------------------|:-------|
| 000_aveda_shampoo          | 71.083 | 001_binder_clips_median      | 77.910 | 002_binder_clips_small       | 70.928 |
| 003_bombik_bucket          | 83.108 | 004_bonne_maman_blueberry    | 66.292 | 005_bonne_maman_raspberry    | 45.002 |
| 006_bonne_maman_strawberry | 38.503 | 007_costa_caramel            | 68.498 | 008_essential_oil_bergamot   | 64.014 |
| 009_garlic_toast_spread    | 46.432 | 010_handcream_avocado        | 55.716 | 011_hb_calcium               | 80.809 |
| 012_hb_grapeseed           | 79.404 | 013_hb_marine_collagen       | 63.965 | 014_hellmanns_mayonnaise     | 74.972 |
| 015_illy_blend             | 71.539 | 016_japanese_finger_cookies  | 82.585 | 017_john_

[03/15 17:44:12 d2.utils.events]:  eta: 0:02:37  iter: 1799  total_loss: 0.664  loss_cls: 0.2586  loss_box_reg: 0.2597  loss_rpn_cls: 0.03008  loss_rpn_loc: 0.1181  time: 0.7913  data_time: 0.3284  lr: 0.002  max_mem: 7021M
[03/15 17:44:28 d2.utils.events]:  eta: 0:02:21  iter: 1819  total_loss: 0.6943  loss_cls: 0.2657  loss_box_reg: 0.2679  loss_rpn_cls: 0.03277  loss_rpn_loc: 0.1239  time: 0.7913  data_time: 0.3451  lr: 0.002  max_mem: 7021M
[03/15 17:44:44 d2.utils.events]:  eta: 0:02:06  iter: 1839  total_loss: 0.7003  loss_cls: 0.2747  loss_box_reg: 0.2687  loss_rpn_cls: 0.03604  loss_rpn_loc: 0.1249  time: 0.7914  data_time: 0.3525  lr: 0.002  max_mem: 7021M
[03/15 17:45:00 d2.utils.events]:  eta: 0:01:50  iter: 1859  total_loss: 0.6919  loss_cls: 0.2589  loss_box_reg: 0.2645  loss_rpn_cls: 0.03023  loss_rpn_loc: 0.1132  time: 0.7914  data_time: 0.3416  lr: 0.002  max_mem: 7021M
[03/15 17:45:16 d2.utils.events]:  eta: 0:01:34  iter: 1879  total_loss: 0.6803  loss_cls: 0.2669  lo

[03/15 17:49:07 d2.evaluation.evaluator]: Inference done 3308/6397. Dataloading: 0.0087 s/iter. Inference: 0.0306 s/iter. Eval: 0.0003 s/iter. Total: 0.0396 s/iter. ETA=0:02:02
[03/15 17:49:12 d2.evaluation.evaluator]: Inference done 3429/6397. Dataloading: 0.0087 s/iter. Inference: 0.0306 s/iter. Eval: 0.0003 s/iter. Total: 0.0396 s/iter. ETA=0:01:57
[03/15 17:49:17 d2.evaluation.evaluator]: Inference done 3550/6397. Dataloading: 0.0087 s/iter. Inference: 0.0307 s/iter. Eval: 0.0003 s/iter. Total: 0.0397 s/iter. ETA=0:01:53
[03/15 17:49:22 d2.evaluation.evaluator]: Inference done 3672/6397. Dataloading: 0.0087 s/iter. Inference: 0.0307 s/iter. Eval: 0.0003 s/iter. Total: 0.0398 s/iter. ETA=0:01:48
[03/15 17:49:27 d2.evaluation.evaluator]: Inference done 3793/6397. Dataloading: 0.0088 s/iter. Inference: 0.0307 s/iter. Eval: 0.0003 s/iter. Total: 0.0398 s/iter. ETA=0:01:43
[03/15 17:49:32 d2.evaluation.evaluator]: Inference done 3913/6397. Dataloading: 0.0088 s/iter. Inference: 0.0308 s

[03/15 17:51:41 d2.engine.defaults]: Evaluation results for coco_realDB_val in csv format:
[03/15 17:51:41 d2.evaluation.testing]: copypaste: Task: bbox
[03/15 17:51:41 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[03/15 17:51:41 d2.evaluation.testing]: copypaste: 70.3013,89.6472,83.4352,60.9623,74.7306,84.9185


## Inference on FasterRCNN

In [6]:
# Datasets
cfg.DATASETS.TEST = ('coco_realDB_test',)  # set test data
cfg.DATALOADER.NUM_WORKERS = 0
cfg.INPUT.MAX_SIZE_TEST = 1024
cfg.INPUT.MIN_SIZE_TEST = 0

# Load trained weights
cfg.MODEL.WEIGHTS = os.path.join("exp_on_FasterRCNN", "model_final.pth")  # path to the model we just trained

from detectron2.engine import DefaultPredictor

predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("coco_realDB_test", cfg, False, output_dir="./output_on_FasterRCNN")
test_loader = build_detection_test_loader(cfg, "coco_realDB_test")
inference_on_dataset(predictor.model, test_loader, evaluator)

[03/15 19:37:52 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from exp_on_FasterRCNN_0/model_FasterRCNN.pth ...
WARNING [03/15 19:37:52 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using configs, this is deprecated behavior. Please pass in explicit arguments instead.
[03/15 19:37:52 d2.data.datasets.coco]: Loaded 160 images in COCO format from /home/SQQ/svid/NeRFDetExps/Data/annotations/instances_test_8.json
[03/15 19:37:52 d2.data.build]: Distribution of instances among all 100 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
| 000_aveda_s.. | 35           | 001_binder_.. | 55           | 002_binder_.. | 50           |
| 003_bombik_.. | 42           | 004_bonne_m.. | 24           | 005_bonne_m.. | 25           |
| 006_bonne_m.. | 22           | 007_costa_c.. | 52           | 008_essenti.. | 41    

/home/SQQ/anaconda3/envs/svid/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/15 19:37:55 d2.evaluation.evaluator]: Inference done 11/160. Dataloading: 0.0070 s/iter. Inference: 0.0294 s/iter. Eval: 0.0002 s/iter. Total: 0.0366 s/iter. ETA=0:00:05
[03/15 19:38:00 d2.evaluation.evaluator]: Inference done 143/160. Dataloading: 0.0083 s/iter. Inference: 0.0293 s/iter. Eval: 0.0002 s/iter. Total: 0.0379 s/iter. ETA=0:00:00
[03/15 19:38:01 d2.evaluation.evaluator]: Total inference time: 0:00:05.829168 (0.037608 s / iter per device, on 1 devices)
[03/15 19:38:01 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.029143 s / iter per device, on 1 devices)
[03/15 19:38:01 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[03/15 19:38:01 d2.evaluation.coco_evaluation]: Saving results to ./output_on_FasterRCNN/coco_instances_results.json
[03/15 19:38:01 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
[03/15 19:38:0

OrderedDict([('bbox',
              {'AP': 19.540672844834543,
               'AP50': 29.205249682984284,
               'AP75': 23.260544483677727,
               'APs': 9.417337681406377,
               'APm': 29.797716613569612,
               'APl': 41.94837340876945,
               'AP-000_aveda_shampoo': 36.885054653867314,
               'AP-001_binder_clips_median': 20.471351482974384,
               'AP-002_binder_clips_small': 28.023668778088666,
               'AP-003_bombik_bucket': 7.916666666666666,
               'AP-004_bonne_maman_blueberry': 1.5023142347515304,
               'AP-005_bonne_maman_raspberry': 6.882088208820882,
               'AP-006_bonne_maman_strawberry': 0.1798256748751798,
               'AP-007_costa_caramel': 34.6542185463134,
               'AP-008_essential_oil_bergamot': 34.628817449160906,
               'AP-009_garlic_toast_spread': 6.03960396039604,
               'AP-010_handcream_avocado': 0.07295466388744137,
               'AP-011_hb_ca